In [31]:
#!pip install python-dotenv
#!pip install openai
#!pip install imageio
#!pip install azure-ai-documentintelligence
#!pip install azure-core

In [32]:
from openai import AzureOpenAI
import dotenv
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, AnalyzeDocumentRequest

dotenv.load_dotenv(override=True)

True

In [33]:
client = AzureOpenAI(
    azure_endpoint=os.getenv('AzureOpenAiEndpoint'),
    api_key=os.getenv('AzureOpenAiKey'),
    azure_deployment='gpt-4o',
    api_version='2024-02-15-preview'
)

In [34]:
documentIntelligenceClient = DocumentIntelligenceClient(
        credential=AzureKeyCredential(key=os.getenv('AzureDiKey')),
        endpoint=os.getenv('AzureDiEndpoint')
        )

In [35]:
with open("table.pdf", "rb") as f:
    documentcontent = f.read()

In [36]:
documentcontent

b'%PDF-1.6\r%\xe2\xe3\xcf\xd3\r\n96 0 obj\r<</Linearized 1/L 57615/O 98/E 51696/N 1/T 57300/H [ 499 197]>>\rendobj\r                  \r\n116 0 obj\r<</DecodeParms<</Columns 4/Predictor 12>>/Filter/FlateDecode/ID[<5BEA3A544E71A946B179AD52FA4531BE><A5D643A71CF98B429D935DF3F91ABED3>]/Index[96 30]/Info 95 0 R/Length 94/Prev 57301/Root 97 0 R/Size 126/Type/XRef/W[1 2 1]>>stream\r\nh\xdebbd\x10``b`Z\x01$\x18\x97\x80\x08\x07 \xc1|\x07$V\tbi\x82X\xf2 \x89\x1d \xae\t\x88\xab\x02"XA\xc4u \xc1\x15\x05\x92U\x06\x12\xaa\x8fA,\x10\xd7c)H\xf6%\x900\xbd\x05$,t\x19\x98\x18\x19\xde\x03Y\x0c\x0c\x8c\x04\x88\xff\x8c\x13~\x02\x04\x18\x00\xf1\xca\x0e2\r\nendstream\rendobj\rstartxref\r\n0\r\n%%EOF\r\n        \r\n125 0 obj\r<</C 115/Filter/FlateDecode/I 137/Length 104/O 99/S 38>>stream\r\nh\xdeb``\x10b``\x9a\xce\x00\x04\xc7\xa60\xa0\x02F fa\xe0h@\x16\x13\x82b\x06Fe\x06~\x06+\xb6\'\\\x1b\x04\x12\xe4\x1bT\x1d:\x1a\x19\x19\x8f0]\xf0\xdc\xc1\xc0R\xc1\xc0\xfb\xd8\x91\xe1Xz\x1e\x93\x04\xdc\xa0\x93\x1f\xa0\x06\xde\

In [37]:
poller = documentIntelligenceClient.begin_analyze_document(
        "prebuilt-layout", AnalyzeDocumentRequest(
            bytes_source=documentcontent,
        ),output_content_format="text"
)

analyzedDocumentContent = poller.result()

In [38]:
print(analyzedDocumentContent)

{'apiVersion': '2024-02-29-preview', 'modelId': 'prebuilt-layout', 'stringIndexType': 'textElements', 'content': 'Example table This is an example of a data table.\nDisability Category\nParticipants\nBallots Completed\nBallots Incomplete/ Terminated\nResults\nAccuracy\nTime to complete\nBlind\n5\n1\n4\n34.5%, n=1\n1199 sec, n=1\nLow Vision\n5\n2\n3\n98.3% n=2 (97.7%, n=3)\n1716 sec, n=3 (1934 sec, n=2)\nDexterity\n5\n4\n1\n98.3%, n=4\n1672.1 sec, n=4\nMobility\n3\n3\n0\n95.4%, n=3\n1416 sec, n=3', 'pages': [{'pageNumber': 1, 'angle': 0, 'width': 8.5, 'height': 11, 'unit': 'inch', 'words': [{'content': 'Example', 'polygon': [0.9909, 1.3685, 1.7671, 1.3667, 1.7653, 1.5673, 0.9909, 1.5658], 'confidence': 0.996, 'span': {'offset': 0, 'length': 7}}, {'content': 'table', 'polygon': [1.8164, 1.3665, 2.2874, 1.364, 2.2874, 1.5705, 1.8145, 1.5675], 'confidence': 0.996, 'span': {'offset': 8, 'length': 5}}, {'content': 'This', 'polygon': [0.991, 1.6081, 1.3629, 1.6063, 1.3554, 1.8057, 0.9825, 1.7

In [39]:
def document_ask(prompt,analyzedDocumentContent):
    response =client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {
                'role': 'user',
                'content': prompt
            },
            {
                'role': 'system',
                'content': f'You know about this file: {analyzedDocumentContent}'
            }
        ]
    )
    return response.choices[0].message.content

In [40]:
document_ask("How many participants are covered?",analyzedDocumentContent)

'The table lists the number of participants across four disability categories. Here are the participant counts:\n\n1. **Blind:** 5 participants\n2. **Low Vision:** 5 participants\n3. **Dexterity:** 5 participants\n4. **Mobility:** 3 participants\n\nTo get the total number of participants covered:\n\n\\[ 5 (Blind) + 5 (Low Vision) + 5 (Dexterity) + 3 (Mobility) = 18 \\]\n\nSo, **18 participants** are covered in the table.'

In [41]:
document_ask("What is the Accuracy in the Results Column for Dexterity?",analyzedDocumentContent)

'In the provided data table, the **Accuracy** in the **Results** column for **Dexterity** is **98.3%** with **n=4**.'

In [42]:
document_ask("What is the Accuracy in the Results Column for Low Vision?",analyzedDocumentContent)

'For the Low Vision category in the given table, the "Accuracy" in the "Results" column is as follows:\n\n- **98.3%** \\(n=2\\)  \n- **(97.7%, n=3)** (The value provided in parentheses)\n\nTherefore, the primary accuracy value is 98.3% with a sample size of 2, and a supplementary or potentially revised accuracy value of 97.7% with a sample size of 3.'

In [43]:
document_ask("What is the Time to complete in the results column for low vision?",analyzedDocumentContent)

'In the provided table, the "Time to complete" results column for participants with low vision indicates two sets of data:\n\n1. **1716 seconds, n=3**: This represents the primary time measurement for three participants.\n2. **1934 seconds, n=2**: This value, shown in parentheses, represents an alternative or secondary time measurement for two participants.\n\nThese numbers suggest that for the low vision category, three participants had a recorded time of 1716 seconds on average to complete the task, and another set of two participants had a longer time of 1934 seconds on average.'

In [30]:
document_ask("Summarize all Participants as well as Ballots Completed. Add both numbers at the end and show the result.",analyzedDocumentContent)

'Here is a summary of the Disability Categories, Participants, and Ballots Completed from the table:\n\n**Disability Category: Blind**\n- Participants: 5\n- Ballots Completed: 1\n\n**Disability Category: Low Vision**\n- Participants: 5\n- Ballots Completed: 2\n\n**Disability Category: Dexterity**\n- Participants: 5\n- Ballots Completed: 4\n\n**Disability Category: Mobility**\n- Participants: 3\n- Ballots Completed: 3\n\nTo compute the total number of Participants and Ballots Completed:\n\n- Total Participants: 5 + 5 + 5 + 3 = 18\n- Total Ballots Completed: 1 + 2 + 4 + 3 = 10\n\n**Result:**\n- Total Participants: 18\n- Total Ballots Completed: 10\n- Combined Total: 18 + 10 = 28'

### How the embedding of one table could look like and some other follow-up questions based on the saved json format of the table

In [27]:
rawtable = document_ask("Output the table structure as Json",analyzedDocumentContent)
print(rawtable)

Here's a JSON representation of the table structure along with its content:

```json
{
  "rowCount": 6,
  "columnCount": 6,
  "cells": [
    {
      "kind": "columnHeader",
      "rowIndex": 0,
      "columnIndex": 0,
      "rowSpan": 2,
      "content": "Disability Category",
      "boundingRegions": [
        {
          "pageNumber": 1,
          "polygon": [1.1756, 1.9957, 2.2408, 1.9957, 2.2343, 2.9874, 1.1691, 2.9874]
        }
      ],
      "spans": [
        {
          "offset": 50,
          "length": 19
        }
      ],
      "elements": [
        "/paragraphs/1"
      ]
    },
    {
      "kind": "columnHeader",
      "rowIndex": 0,
      "columnIndex": 1,
      "rowSpan": 2,
      "content": "Participants",
      "boundingRegions": [
        {
          "pageNumber": 1,
          "polygon": [2.2408, 1.9957, 3.2469, 1.9957, 3.2469, 2.9874, 2.2343, 2.9874]
        }
      ],
      "spans": [
        {
          "offset": 70,
          "length": 12
        }
      ],
     

In [28]:
document_ask("How many participants are in the study?",rawtable)

'The study includes the following number of participants from each disability category:\n\n- Blind: 5 participants\n- Low Vision: 5 participants\n- Dexterity: 5 participants\n- Mobility: 3 participants\n\nTotal Participants:\n5 (Blind) + 5 (Low Vision) + 5 (Dexterity) + 3 (Mobility) = 18 participants\n\nSo, the study has a total of 18 participants.'

In [29]:
document_ask("How many participants are in the study with Mobility?",rawtable)

'In the study, there are 3 participants categorized under "Mobility."'